In [2]:
import statsmodels.api as sm
import scipy as sp
import scipy.linalg as spla
import scipy.optimize as spopt
import scipy.stats as spst
import numpy as np
import numpy.linalg as npla
import pandas as pd
from scipy.stats.distributions import t
from scipy.optimize import fmin, minimize
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [3]:
dat = sm.datasets.get_rdataset("Guerry", "HistData").data

### Assignment 1 (2 points).
Write a function assignment_1 that replicates the outcome of assignment_1_true on the *dat* dataset from the beginning of the notebook

In [15]:
smfOLS = sm.regression.linear_model.OLS.from_formula
def assignment_1_true(formula, data):
    
    fit = smfOLS(formula, data = data).fit()
    
    return fit.conf_int()

In [3]:
def assignment_1(formula, data):
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    X = data[xcolumns].values
    X = sm.add_constant(X)
    #coeffs
    betas = spla.inv(X.T@X)@(X.T@Y)
    #sigma squared
    sigma2 = sum((Y - X@betas)**2)/(len(Y)-6)
    #covs
    covs = sigma2*spla.inv(X.T@X)
    #standart errors 
    se = np.sqrt(np.diag(covs))
    #z_value for aplha = 5%
    alpha = 0.05 
    z = t.ppf(1 - alpha/2, len(X)-len(betas))
    results = np.round([[betas[i]-z*se[i], betas[i]+z*se[i]] for i in range(len(betas))], 3)
    
    df = pd.DataFrame(results, columns = [0,1])
    df.index = np.array(['Intercept'] + xcolumns)
    
    return df
assignment_1('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', dat)

,0,1
Intercept,-6.027,41.047
Literacy,-0.479,0.180
Donations,-0.001,0.001
Infants,-0.000,0.001
Wealth,0.096,0.517
Commerce,-0.101,0.405


### Assignment 2 (3 points).
- Write a function that finds the coefficients for the **elastic net** regularized ols with coefficients $\lambda$ and $\mu$ of your choice
- This time you do not need to find the standard errors

$$L = ||Y - X \beta ||^2 + \lambda ||\beta||^2 + \mu |\beta|_0\to \min$$.

In [4]:
def assignment_2(formula, data, mu, lamda):
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    X = data[xcolumns].values
    X = sm.add_constant(X)
    covs = 6
    def loss(Y, X, beta):
        z = Y - X@beta
        return z@z.T + lamda*(beta@beta.T) + mu*sum(np.ones(covs)*beta)
    bounds = [(None, None)]*covs
    beta = np.round(spopt.shgo(lambda b: loss(Y, X, b), bounds).x, 2)    
    
    df = pd.DataFrame(beta, columns = [0])
    df.index = np.array(['Intercept'] + xcolumns)
    
    return df

assignment_2('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', dat, 2, 2)

,0
Intercept,10.46
Literacy,-0.07
Donations,0.00
Infants,0.00
Wealth,0.31
Commerce,0.19


### Assignment 3 (5 points)
- Write a function that finds the coefficients for the **elastic net** regularized ols with crossvalidation
- Use number of folds of your choice
- You do not need to find the standard errors
- Search for $\lambda$, $\mu$ in the range [0,5]x[0,5]

In [11]:
def assignment_3(formula, data, folds):
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    X = data[xcolumns].values
    X = sm.add_constant(X)
#K folds 
    kf = KFold(n_splits=folds)
#Lambd's and Mu's
    ls = np.linspace(0,5,100)
    ms = np.linspace(0,5,100)
    covs = 6
#define loss functions for elastic net
    def loss(Y, X, beta, lamda, mu):
            covs = 6
            z = Y - X@beta
            return z@z.T + lamda*(beta@beta.T) + mu*sum(np.ones(covs)*beta)
    def run(lamda, mu):
        avg = 0
        
        #split in 5 kfolds
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
        #compute betas vector
            
            bounds = [(None, None)] * covs 
            beta = np.round(spopt.shgo(lambda B: loss(y_train, X_train, B, lamda, mu), bounds).x, 2)
        #compute sum of squared residuals 
            errs = y_test - X_test@beta
            SSR = errs@errs.T
            avg += SSR
        return SSR/folds
    vals = []
    for l in ls:
        for m in ms:
            ssr = run(l, m)
            vals.append([l, m, ssr])
    min_L_Mu_SSR = vals[np.array(vals).argmin(0)[-1]]
    bounds = [(None, None)] * covs 
    beta = np.round(spopt.shgo(lambda B: loss(Y, X, B, 0,0), bounds).x, 2)
        
    df = pd.DataFrame(beta, columns = [0])
    df.index = np.array(['Intercept'] + xcolumns)

    
    return df, min_L_Mu_SSR
assignment_3('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', dat, 10)

(               0
 Intercept  17.51
 Literacy   -0.15
 Donations   0.00
 Infants     0.00
 Wealth      0.31
 Commerce    0.15,
 [0.7575757575757576, 3.3333333333333335, 606.2610699999999])